# Work with Azure Blob Storage

To start, ensure that you have a Azure Blob Storage account. Reference the documentation on how to create a storage account if needed. [Create a storage account](https://learn.microsoft.com/azure/storage/common/storage-account-create?tabs=azure-portal)

Also, have a container created in the Azure Blob Storage account. You can use this quick-start to help guide you in creating a container. [Quickstart: Upload, download, and list blobs with the Azure portal](https://learn.microsoft.com/azure/storage/blobs/storage-quickstart-blobs-portal)

Blob Storage REST endpoint are documented here: [Azure Blob Storage REST API](https://learn.microsoft.com/rest/api/storageservices/blob-service-rest-api). 
The samples use the [Get Blob](https://learn.microsoft.com/rest/api/storageservices/get-blob?tabs=azure-ad), [Put Blob](https://learn.microsoft.com/en-us/rest/api/storageservices/put-blob?tabs=azure-ad), and the (Create Container)[https://learn.microsoft.com/rest/api/storageservices/create-container?tabs=azure-ad] APIs.

The following samples assume that there is an Azure Blob Storage Account deployed at `https://blobby.blob.core.windows.net/` and a container named `myblobs`. To have the samples working in your environment, make sure adjust URL and container name so that they will match your account.

## Work with files in Azure Blob Storage using a SAS Token

Only authenticated requests can send events to Event Hubs. One way to authenticate a request is to provide a Shared Access Signature token: 
- [Authorizing access to Event Hubs resources using Shared Access Signatures](https://learn.microsoft.com/en-us/azure/event-hubs/authorize-access-shared-access-signature)
- [Generate SAS token](https://learn.microsoft.com/en-us/rest/api/eventhub/generate-sas-token). 

At the moment is not possible to generate a SAS token directly from Azure SQL database, but you can put the code for generating such a token in an Azure Function and call it from Azure SQL database using `sp_invoke_external_rest_point` as well.

Once you have the token you can add it into a Database Scoped Credential:

In [ ]:
-- make sure a database master key exists
if not exists(select * from sys.symmetric_keys where [name] = '##MS_DatabaseMasterKey##') begin
    create master key encryption by password = 'LONg_Pa$$_w0rd!'
end

-- create database scoped credential
create database scoped credential [https://azure-event-hubs.servicebus.windows.net]
with identity = 'HTTPEndpointHeaders', 
secret = '{"Authorization": "SharedAccessSignature sr=azure-event-hubs.servicebus.windows.net%2fmyeventhub&sig=RVDJM1cSo71j73%2bWR0t7ZCZukIjMEvBn%2bWWqSlqkJeM%3d&se=1697310598&skn=RootManageSharedAccessKey"}';

You can then send messages to Event Hubs using the "Send Event" API, which is available at `https://azure-event-hubs.servicebus.windows.net/myeventhub/messages` :

In [ ]:
declare @payload nvarchar(max) = '{"UserId": "6C5E29A2-A5E7-449D-BD14-259D61ADC6BE", "FirstName": "John", "LastName": "Doe"}';
declare @headers nvarchar(4000) = N'{"BrokerProperties": "' + string_escape('{"PartitionKey": "6C5E29A2-A5E7-449D-BD14-259D61ADC6BE"}', 'json') + '"}'
declare @ret int, @response nvarchar(max)

exec @ret = sp_invoke_external_rest_endpoint 
        @url = 'https://azure-event-hubs.servicebus.windows.net/myeventhub/messages',
        @headers = @headers,
        @payload = @payload,
		@credential = [https://azure-event-hubs.servicebus.windows.net],
        @response = @response output;

select @response;

## Send Events using Managed Identities

Follow the instructions here: [Enable Managed Identity in Azure SQL](./azure-sql-enable-msi.ipynb) to make sure you have Managed Identity enabled for your Azure SQL database, and then check how to grant to right permission on Event Hubs to the Azure SQL Manage Identity, following the instructions here: [Grant permissions to a managed identity in Azure AD](https://learn.microsoft.com/azure/event-hubs/authenticate-managed-identity?tabs=latest#grant-permissions-to-a-managed-identity-in-azure-ad).

Once that is done you just need to create a Database Scoped Credentials with the string `Managed Identity` as identity and `https://eventhubs.azure.net` as the `resourceid`:

In [ ]:
-- make sure a database master key exists
if not exists(select * from sys.symmetric_keys where [name] = '##MS_DatabaseMasterKey##') begin
    create master key encryption by password = 'LONg_Pa$$_w0rd!'
end

-- create database scoped credential
if exists(select * from sys.database_scoped_credentials where [name] = 'https://azure-event-hubs.servicebus.windows.net') begin
    drop database scoped credential [https://azure-event-hubs.servicebus.windows.net];
end;
create database scoped credential [https://azure-event-hubs.servicebus.windows.net]
with identity = 'Managed Identity', 
secret = '{"resourceid": "https://eventhubs.azure.net" }';

Once this is done you can send the message using the same code as before:

In [ ]:
declare @payload nvarchar(max) = '{"UserId": "6C5E29A2-A5E7-449D-BD14-259D61ADC6BE", "FirstName": "John", "LastName": "Doe"}';
declare @headers nvarchar(4000) = N'{"BrokerProperties": "' + string_escape('{"PartitionKey": "6C5E29A2-A5E7-449D-BD14-259D61ADC6BE"}', 'json') + '"}'
declare @ret int, @response nvarchar(max)

exec @ret = sp_invoke_external_rest_endpoint 
        @url = 'https://azure-event-hubs.servicebus.windows.net/myeventhub/messages',
        @headers = @headers,
        @payload = @payload,
		@credential = [https://azure-event-hubs.servicebus.windows.net],
        @response = @response output;

select @response;